In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation, BatchNormalization
from keras.layers import Dropout, Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Reshape
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np

Using TensorFlow backend.


In [2]:
training = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
target = 'label'
trainSplitRandomSeed = 1

In [3]:
X_training = training.loc[:, training.columns != target]
X_training = X_training / 255.0
test = test / 255.0
y_training = training[target]

In [4]:
X_train, X_validation, y_train, y_validation = train_test_split(X_training, y_training, stratify = y_training,
                                                                random_state = trainSplitRandomSeed, test_size=0.1)

In [5]:
X_train = np.array(X_train).reshape(X_train.shape[0], 28, 28, 1)
X_validation = np.array(X_validation).reshape(X_validation.shape[0], 28, 28, 1)
test = np.array(test).reshape(test.shape[0], 28, 28, 1)

y_train = np_utils.to_categorical(y_train, 10)
y_validation = np_utils.to_categorical(y_validation, 10)

In [6]:
def CreateNNModel(dropoutRate):
    model = Sequential()

    model.add(Conv2D(32, (3, 3), input_shape=(28,28,1)))
    model.add(BatchNormalization(axis=-1))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(BatchNormalization(axis=-1))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(64,(3, 3)))
    model.add(BatchNormalization(axis=-1))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(BatchNormalization(axis=-1))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Flatten())

    # Fully connected layer
    model.add(Dense(512))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(dropoutRate))
    model.add(Dense(10))

    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [7]:
nnRegressor = KerasClassifier(build_fn=CreateNNModel, verbose=1)

estimators = [('nn', nnRegressor)]

pipeline = Pipeline(estimators)
param_grid = {
  'nn__dropoutRate': [0.2],
  'nn__batch_size' : [500],
  'nn__validation_split' : [0.1],
  'nn__epochs' : [20]
}

In [8]:
reg = GridSearchCV(pipeline, param_grid, cv=3)

In [9]:
reg.fit(X_train, y_train)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 22680 samples, validate on 2520 samples
Epoch 1/20
22680/22680 [==============================] - 5s 203us/step - loss: 0.2516 - acc: 0.9222 - val_loss: 0.1337 - val_acc: 0.9607
Epoch 2/20
22680/22680 [==============================] - 2s 89us/step - loss: 0.0481 - acc: 0.9859 - val_loss: 0.0541 - val_acc: 0.9833
Epoch 3/20
22680/22680 [==============================] - 2s 89us/step - loss: 0.0261 - acc: 0.9937 - val_loss: 0.0542 - val_acc: 0.9841
Epoch 4/20
22680/22680 [==============================] - 2s 89us/step - loss: 0.0166 - acc: 0.9972 - val_loss: 0.0507 - val_acc: 0.9853
Epoch 5/20
22680/22680 [==============================] - 2s 88us/step - loss: 0.0103 - acc: 0.9985 - val_loss: 0.0479 - val_acc: 0.9853
Epoch 6/20
22680/22680 [======

22680/22680 [==============================] - 2s 90us/step - loss: 0.0017 - acc: 1.0000 - val_loss: 0.0328 - val_acc: 0.9901
Epoch 11/20
22680/22680 [==============================] - 2s 91us/step - loss: 0.0013 - acc: 1.0000 - val_loss: 0.0346 - val_acc: 0.9881
Epoch 12/20
22680/22680 [==============================] - 2s 91us/step - loss: 0.0010 - acc: 1.0000 - val_loss: 0.0306 - val_acc: 0.9897
Epoch 13/20
22680/22680 [==============================] - 2s 92us/step - loss: 9.1826e-04 - acc: 1.0000 - val_loss: 0.0297 - val_acc: 0.9897
Epoch 14/20
22680/22680 [==============================] - 2s 91us/step - loss: 7.4902e-04 - acc: 1.0000 - val_loss: 0.0309 - val_acc: 0.9905
Epoch 15/20
22680/22680 [==============================] - 2s 91us/step - loss: 6.6358e-04 - acc: 1.0000 - val_loss: 0.0294 - val_acc: 0.9901
Epoch 16/20
22680/22680 [==============================] - 2s 90us/step - loss: 5.7992e-04 - acc: 1.0000 - val_loss: 0.0290 - val_acc: 0.9909
Epoch 17/20
22680/22680 [=====

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('nn', <keras.wrappers.scikit_learn.KerasClassifier object at 0x000002006F50C278>)]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'nn__dropoutRate': [0.2], 'nn__batch_size': [500], 'nn__validation_split': [0.1], 'nn__epochs': [20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [10]:
reg.score(X_validation, y_validation)

4200/4200 [==============================] - 0s 34us/step


0.9914285739262899

In [14]:
result = reg.predict(test)
resultDf = pd.DataFrame(data=result.flatten())
resultDf.columns = ['Label']
resultDf = pd.DataFrame(data=result.flatten())
resultDf.index = np.arange(1, len(resultDf) + 1)
resultDf.to_csv('result.txt', header = ['Label'], index_label = 'ImageId')

28000/28000 [==============================] - 1s 33us/step
